In [ ]:
import torch
import numpy as np
import torch.nn as nn

In [ ]:
!wget 'https://raw.githubusercontent.com/r0ckYr/GenerativeAI/main/text.txt'

In [ ]:
with open('text.txt', 'r') as f:
  input = f.read()

In [ ]:
input = input[:1000]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn

# Sample vocabulary and dataset
vocab = ['<PAD>', 'hello', 'world', 'how', 'are', 'you']
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
sequences = [
    [1, 2, 3, 4],
    [2, 3, 4, 5],
    [3, 4, 5, 0]  # Adding padding token <PAD> at the end
]

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded.unsqueeze(1), hidden)  # Adding unsqueeze to match sequence length
        output = self.fc(output)
        return output, hidden

# Training
def train_model(model, sequences, word_to_idx, idx_to_word, num_epochs=10, learning_rate=0.01):
    vocab_size = len(word_to_idx)
    embedding_dim = 10
    hidden_dim = 20

    model = model(vocab_size, embedding_dim, hidden_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0
        for sequence in sequences:
            input_seq = torch.tensor(sequence[:-1], dtype=torch.long, device=device)
            target_seq = torch.tensor(sequence[1:], dtype=torch.long, device=device)

            hidden = None
            optimizer.zero_grad()

            output, hidden = model(input_seq, hidden)
            loss = criterion(output.view(-1, vocab_size), target_seq)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

if __name__ == '__main__':
    model = GRUModel
    train_model(model, sequences, word_to_idx, idx_to_word, num_epochs=10, learning_rate=0.01)


Epoch [1/10], Loss: 5.5728
Epoch [2/10], Loss: 4.9335
Epoch [3/10], Loss: 4.3446
Epoch [4/10], Loss: 3.6880
Epoch [5/10], Loss: 2.9815
Epoch [6/10], Loss: 2.3113
Epoch [7/10], Loss: 1.7484
Epoch [8/10], Loss: 1.2927
Epoch [9/10], Loss: 0.9247
Epoch [10/10], Loss: 0.6444


In [ ]:
import torch
import torch.nn.functional as F

def generate_text(model, start_word, word_to_idx, idx_to_word, max_length=20):
    device = next(model.parameters()).device  # Get the device from the model parameters

    model.eval()

    with torch.no_grad():
        input_word = torch.tensor([word_to_idx[start_word]], dtype=torch.long, device=device)
        hidden = None
        generated_text = [start_word]

        for _ in range(max_length):
            input_word = input_word.to(device)  # Move input tensor to the correct device
            output, hidden = model(input_word, hidden)
            output_probs = F.softmax(output, dim=2).squeeze().cpu().numpy()
            next_word_idx = np.random.choice(len(output_probs), p=output_probs)
            input_word.fill_(next_word_idx)
            generated_text.append(idx_to_word[next_word_idx])

            if idx_to_word[next_word_idx] == '<PAD>':
                break

        return ' '.join(generated_text)

if __name__ == '__main__':
    model = GRUModel(len(vocab), embedding_dim=10, hidden_dim=20)
    generated_text = generate_text(model, start_word='world', word_to_idx=word_to_idx, idx_to_word=idx_to_word)
    print(generated_text)


world hello <PAD>
